In [1]:
import requests
import xml.etree.ElementTree as ET
import csv
import time


## API KEYS

In [2]:
TOMTOM_API_KEY = 'xdmIEARrmlgFyJkuN2k0LyIkLHAwBRfn'

In [3]:
CTA_API_KEY = 'ccbbacb6affa4dfdb45f796b2c369773'

SWAPS for TOMTOM = U43oanQyySyqXVfYZAb0t2zgcdHgAVE0

## MAP ID VALUES

In [4]:
 mapid_values = ['40260', '40380', '41700', '40680', '40850', 
                 '40160', '40040', '40730', '40460', '40330', 
                 '41400', '40350', '41160', '41220', '41120', 
                 '41320', '40820', '40960', '41020']

## LOOP CTA TRAINS AND ABOVE-GROUND TRAFFIC SCRIPT

In [ ]:
def fetch_traffic_speed(latitude, longitude):
    traffic_url = 'https://api.tomtom.com/traffic/services/4/flowSegmentData/relative/10/json'
    traffic_params = {
        'point': f'{latitude},{longitude}',
        'unit': 'mph',
        'key': TOMTOM_API_KEY
    }
    
    try:
        traffic_response = requests.get(traffic_url, params=traffic_params)
        traffic_data = traffic_response.json()
        
        if traffic_response.status_code == 200 and "flowSegmentData" in traffic_data:
            flow_segment_data = traffic_data["flowSegmentData"]
            free_flow_speed_mph = flow_segment_data.get("freeFlowSpeed", "N/A")
            current_speed_mph = flow_segment_data.get("currentSpeed", "N/A")
            return free_flow_speed_mph, current_speed_mph
        else:
            print("TomTom API request error - status code:", traffic_response.status_code)
            return None, None
    except requests.exceptions.RequestException as e:
        print("Request error:", e)
        return None, None

def fetch_CTA_and_traffic():
    cta_url = 'https://lapi.transitchicago.com/api/1.0/ttarrivals.aspx'

    while True:
        csv_file = open('combined_data.csv', 'a', newline='', encoding='utf-8')
        csv_writer = csv.writer(csv_file)
        
        for mapid in mapid_values:
            for _ in range(1):
                cta_params = {
                    'mapid': mapid,
                    'max': '7', 
                    'key': CTA_API_KEY
                }
                
                cta_response = requests.get(cta_url, params=cta_params)
                
                if cta_response.status_code == 200:
                    try:
                        root = ET.fromstring(cta_response.text)
                        eta_elements = root.findall(".//eta")

                        for eta in eta_elements:
                            route = eta.find("rt").text
                            prd_time = eta.find("prdt").text
                            arr_time = eta.find("arrT").text
                            sch_bool = eta.find("isSch").text
                            flt_bool = eta.find("isFlt").text
                            dly_bool = eta.find("isDly").text
                            station_name = eta.find("staNm").text
                            dest_name = eta.find("destNm").text

                            latitude_element = eta.find("lat")
                            longitude_element = eta.find("lon")

                            if latitude_element is not None and longitude_element is not None:
                                latitude = latitude_element.text
                                longitude = longitude_element.text

                                if latitude and longitude:
                                    free_flow_speed, current_speed = fetch_traffic_speed(float(latitude), float(longitude))

                                    if free_flow_speed is not None:
                                        csv_writer.writerow([mapid, route, prd_time, arr_time, sch_bool, flt_bool, dly_bool, station_name, dest_name, latitude, longitude, free_flow_speed, current_speed])
                                else:
                                    print("Invalid latitude or longitude in CTA API response.")
                            else:
                                print("Missing latitude or longitude in CTA API response.")

                    except ET.ParseError as e:
                        print("Error parsing CTA API XML:", e)
                        print("Response content:", cta_response.text)
                else:
                    print("CTA API request error - status code:", cta_response.status_code)
                    print("Response content:", cta_response.text)

        csv_file.close()

        time.sleep(10 * 60)  # x minutes in seconds

fetch_CTA_and_traffic()


Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":15,"freeFlowSpeed":15,"currentTravelTime":24,"freeFlowTravelTime":24,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.873118298114612,"longitude":-87.627485605801326},{"latitude":41.873469686444352,"longitude":-87.627492311323863},{"latitude":41.873664166259687,"longitude":-87.627497675741893},{"latitude":41.873884109984736,"longitude":-87.627501699055415},{"latitude":41.874077215457625,"longitude":-87.627507063473445},{"latitude":41.874369556310938,"longitude":-87.627513768996010},{"latitude":41.874589497608639,"longitude":-87.627520474518548}]},"@version":"traffic-service-flow 1.0.095"}}
Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":14,"freeFlowSpeed":14,"currentTravelTime":39,"freeFlowTravelTime":39,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.878084428024316,"longitude":-87.635205003347991},{"latitude":41.87809

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":24,"freeFlowSpeed":24,"currentTravelTime":75,"freeFlowTravelTime":75,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.851337439648162,"longitude":-87.744271668545963},{"latitude":41.851328074398474,"longitude":-87.744786652676950},{"latitude":41.851326700828423,"longitude":-87.744876506678963},{"latitude":41.851322704988064,"longitude":-87.745103153340779},{"latitude":41.851318646712421,"longitude":-87.745336505525117},{"latitude":41.851310592595460,"longitude":-87.745851489656104},{"latitude":41.851305223183530,"longitude":-87.746209564559663},{"latitude":41.851302538477427,"longitude":-87.746378543727644},{"latitude":41.851298542635561,"longitude":-87.746709796541069},{"latitude":41.851287803809313,"longitude":-87.747341456764232},{"latitude":41.851286492673403,"longitude":-87.747476908319499},{"latitude":41.851269010859028,"longitude":-87.748588683956442},{"latitude":41.851265

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":11,"freeFlowSpeed":11,"currentTravelTime":177,"freeFlowTravelTime":177,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.879460404190858,"longitude":-87.627646538342262},{"latitude":41.880079983204361,"longitude":-87.627663972700859},{"latitude":41.880749169220316,"longitude":-87.627684089268499},{"latitude":41.881324552507017,"longitude":-87.627714934672170},{"latitude":41.881541786433111,"longitude":-87.627724322403722},{"latitude":41.882043337494437,"longitude":-87.627741756762319},{"latitude":41.882620019621164,"longitude":-87.627753826702886},{"latitude":41.883196696543244,"longitude":-87.627773943270512},{"latitude":41.883549402339078,"longitude":-87.627779307688542},{"latitude":41.883782791161842,"longitude":-87.627781989897557},{"latitude":41.883981233394017,"longitude":-87.627808811987705},{"latitude":41.884524387848380,"longitude":-87.627822223032780},{"latitude":41.8847

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":15,"freeFlowSpeed":15,"currentTravelTime":24,"freeFlowTravelTime":24,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.873118298114612,"longitude":-87.627485605801326},{"latitude":41.873469686444352,"longitude":-87.627492311323863},{"latitude":41.873664166259687,"longitude":-87.627497675741893},{"latitude":41.873884109984736,"longitude":-87.627501699055415},{"latitude":41.874077215457625,"longitude":-87.627507063473445},{"latitude":41.874369556310938,"longitude":-87.627513768996010},{"latitude":41.874589497608639,"longitude":-87.627520474518548}]},"@version":"traffic-service-flow 1.0.095"}}
Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":11,"freeFlowSpeed":11,"currentTravelTime":177,"freeFlowTravelTime":177,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.879460404190858,"longitude":-87.627646538342262},{"latitude":41.880

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC0","currentSpeed":65,"freeFlowSpeed":65,"currentTravelTime":111,"freeFlowTravelTime":111,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.846849531341810,"longitude":-87.650439626466920},{"latitude":41.846841069133269,"longitude":-87.650522050513970},{"latitude":41.846804380104174,"longitude":-87.650804931366295},{"latitude":41.846729401174720,"longitude":-87.651240226721697},{"latitude":41.846668346319582,"longitude":-87.651554535012323},{"latitude":41.846583878758928,"longitude":-87.651902515955172},{"latitude":41.846513485478980,"longitude":-87.652167175044127},{"latitude":41.846491524034207,"longitude":-87.652253104248203},{"latitude":41.846361777957469,"longitude":-87.652700645341454},{"latitude":41.846238976901084,"longitude":-87.653051743583674},{"latitude":41.846098031126871,"longitude":-87.653421829588069},{"latitude":41.845606988058265,"longitude":-87.654617567485943},{"latitude":41.8455

...

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":15,"freeFlowSpeed":15,"currentTravelTime":122,"freeFlowTravelTime":122,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.852206466414763,"longitude":-87.675869974232256},{"latitude":41.852199785960892,"longitude":-87.676175746060011},{"latitude":41.852199785960892,"longitude":-87.676226708031308},{"latitude":41.852198412409528,"longitude":-87.676273646689097},{"latitude":41.852193043072184,"longitude":-87.676575395203329},{"latitude":41.852182304396102,"longitude":-87.677075627184720},{"latitude":41.852175623939708,"longitude":-87.677435043192801},{"latitude":41.852159515920533,"longitude":-87.678301396704811},{"latitude":41.852143407897302,"longitude":-87.679823550321117},{"latitude":41.852138100989507,"longitude":-87.680095794536186},{"latitude":41.852131358094297,"longitude":-87.680420341827073},{"latitude":41.852124677632560,"longitude":-87.680779757835154},{"latitude":41.8521

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC0","currentSpeed":71,"freeFlowSpeed":71,"currentTravelTime":84,"freeFlowTravelTime":84,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.778039402609068,"longitude":-87.628232686797659},{"latitude":41.778037439150069,"longitude":-87.628230593536173},{"latitude":41.778093924445685,"longitude":-87.628281034061814},{"latitude":41.778237315802620,"longitude":-87.628403295069674},{"latitude":41.778309215677105,"longitude":-87.628460129524456},{"latitude":41.778383139716468,"longitude":-87.628520360441200},{"latitude":41.778416972440930,"longitude":-87.628543361940871},{"latitude":41.778490791558113,"longitude":-87.628595882804845},{"latitude":41.778520243300171,"longitude":-87.628617106044629},{"latitude":41.778665141048087,"longitude":-87.628708041989071},{"latitude":41.778793812845308,"longitude":-87.628778806641847},{"latitude":41.778941237583140,"longitude":-87.628848284148418},{"latitude":41.779054

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":14,"freeFlowSpeed":14,"currentTravelTime":112,"freeFlowTravelTime":112,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.939866409036696,"longitude":-87.658976080748019},{"latitude":41.939873142675324,"longitude":-87.658609959217401},{"latitude":41.939873142675324,"longitude":-87.658467802139569},{"latitude":41.939871771008249,"longitude":-87.658353808256422},{"latitude":41.939873142675324,"longitude":-87.658238473268753},{"latitude":41.939879813964751,"longitude":-87.657715442510707},{"latitude":41.939893218889992,"longitude":-87.657103898855183},{"latitude":41.939905314494517,"longitude":-87.656519177289795},{"latitude":41.939910676462823,"longitude":-87.655934455724420},{"latitude":41.939917410096740,"longitude":-87.655321570964375},{"latitude":41.939930815014122,"longitude":-87.654718073935868},{"latitude":41.939941538945966,"longitude":-87.654090437026241},{"latitude":41.9399

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC0","currentSpeed":67,"freeFlowSpeed":67,"currentTravelTime":33,"freeFlowTravelTime":33,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.983179480071179,"longitude":-87.831047019155349},{"latitude":41.983178138503725,"longitude":-87.830365796710453},{"latitude":41.983165581238566,"longitude":-87.829517218090317},{"latitude":41.983008436798670,"longitude":-87.826443712564213},{"latitude":41.983006041150993,"longitude":-87.826401741195923},{"latitude":41.983000250814840,"longitude":-87.826341839886965},{"latitude":41.982982070169285,"longitude":-87.826107373941667},{"latitude":41.982972926958368,"longitude":-87.825949474803593},{"latitude":41.982912184343242,"longitude":-87.825350900181675},{"latitude":41.982829418591251,"longitude":-87.824534551420300},{"latitude":41.982810961152026,"longitude":-87.824390608047736},{"latitude":41.982739057132683,"longitude":-87.823804503033102},{"latitude":41.982724

Traffic API Response Content: {"flowSegmentData":{"frc":"FRC4","currentSpeed":12,"freeFlowSpeed":12,"currentTravelTime":49,"freeFlowTravelTime":49,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.867533960249617,"longitude":-87.624198558652822},{"latitude":41.867520540124090,"longitude":-87.624881180847268},{"latitude":41.867519229320976,"longitude":-87.624993833625922},{"latitude":41.867519229320976,"longitude":-87.625310334289750},{"latitude":41.867520540124090,"longitude":-87.625381412828673},{"latitude":41.867525908174649,"longitude":-87.625818612898200},{"latitude":41.867521913346366,"longitude":-87.625919195736273},{"latitude":41.867508493218317,"longitude":-87.626329573715651},{"latitude":41.867507119995743,"longitude":-87.626450273121350},{"latitude":41.867505809192394,"longitude":-87.626533421600826},{"latitude":41.867504435969742,"longitude":-87.626593771303675},{"latitude":41.867505809192394,"longitude":-87.626663508738091},{"latitude":41.867501